In [11]:
#Import Ipython that is used for display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [12]:
#Open-source Python package for exploring, visualizing, and analyzing EEGs
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

import torch.nn.functional as F

import os, os.path
from os import listdir

%matplotlib inline

In [13]:
# dataset_path = "../chb-mit/"
dataset_path = './chb-mit/'
# spectrograms_path = "../spectrograms/"
spectrograms_path = './spectrograms/'
samples_path = "./samples/"


if not os.path.exists(samples_path):
    os.makedirs(samples_path)
    
path_train_0 = samples_path + "train/0/"
path_train_1 = samples_path + "train/1/"
path_val_0 = samples_path + "validation/0/"
path_val_1 = samples_path + "validation/1/"
path_test_0 = samples_path + "test/0/"
path_test_1 = samples_path + "test/1/"

#en segundos
window_size = 60
overlap = 10
preictal = 20 * 60

In [14]:
"""
    Canales seleccionados para ser analizados
"""

#Temporal# channels = ['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8-0', 'P8-O2', 'FZ-CZ', 'CZ-PZ']
channels = ['FP1-F7', 'F7-T7', 'T7-P7']

n_channels = len(channels)

print(n_channels)


3


In [15]:
class EEGDataset_train(Dataset):
    def __init__(self):
        self.path0 = path_train_0
        self.path1 = path_train_1
        self.n0 = len([name for name in os.listdir(path_train_0) if os.path.isfile(os.path.join(path_train_0, name))]) 
        self.n1 = len([name for name in os.listdir(path_train_1) if os.path.isfile(os.path.join(path_train_1, name))])
    def __getitem__(self, index):
        if index < self.n0:
            file = np.load(self.path0+"img"+str(index)+".npy")
            y = 0
        else:
            file = np.load(self.path1+"img"+str(index-self.n0)+".npy")
            y = 1

        x = file.astype(np.float32)
        
        x = torch.from_numpy(x)
        
        return x, y
    def __len__(self):
        return self.n0 + self.n1
    
"""class EEGDataset_validation(Dataset):
    def __init__(self):
        self.path0 = path_val_0
        self.path1 = path_val_1
    def __getitem__(self, index):
        if index <= 500:
            file = np.load(self.path0+"img"+str(index+5000)+".npy")
            y = 0
        else:
            file = np.load(self.path1+"img"+str(index+5000-500)+".npy")
            y = 1

        x = file.astype(np.float32)
        
        x = torch.from_numpy(x)
        
        return x, y
    def __len__(self):
        return 1000"""
    
class EEGDataset_test(Dataset):
    def __init__(self):
        self.path0 = path_test_0
        self.path1 = path_test_1
        self.n0 = len([name for name in os.listdir(path_test_0) if os.path.isfile(os.path.join(path_test_0, name))]) 
        self.n1 = len([name for name in os.listdir(path_test_1) if os.path.isfile(os.path.join(path_test_1, name))])
    def __getitem__(self, index):
        if index < self.n0:
            file = np.load(self.path0+"img"+str(index)+".npy")
            y = 0
        else:
            file = np.load(self.path1+"img"+str(index-self.n0)+".npy")
            y = 1

        x = file.astype(np.float32)
        
        x = torch.from_numpy(x)
        
        return x, y
    def __len__(self):
        return self.n0 + self.n1

In [16]:
print(len([name for name in os.listdir(path_test_0) if os.path.isfile(os.path.join(path_test_0, name))]) )
print(len([name for name in os.listdir(path_test_1) if os.path.isfile(os.path.join(path_test_1, name))]) )
print(len([name for name in os.listdir(path_train_0) if os.path.isfile(os.path.join(path_train_0, name))]) )
print(len([name for name in os.listdir(path_train_1) if os.path.isfile(os.path.join(path_train_1, name))]) )

342
342
1512
1512


In [17]:
dataset_train = EEGDataset_train()
#dataset_validation = EEGDataset_validation()
dataset_test = EEGDataset_test()
loader_train = DataLoader(dataset=dataset_train, batch_size=16, shuffle=True, )
#loader_validation = DataLoader(dataset=dataset_validation, batch_size=64, shuffle=True, )
loader_test = DataLoader(dataset=dataset_test, batch_size=16, shuffle=True, )


"""datatiter = iter(loader_train)
data = datatiter.next()
features, labels = data
print(features, labels)"""

'datatiter = iter(loader_train)\ndata = datatiter.next()\nfeatures, labels = data\nprint(features, labels)'

In [18]:
#Selecciona cuda si está disponible
USE_GPU = True
dtype = torch.float32
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('using device:', device)

using device: cpu


In [19]:
# The following function is provided to check the accuracy of the 
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()  
    with torch.no_grad(): # We need to manually specify that no gradient needs to be computed
        for x, y in loader: # Get data from data loader
            x = x.to(device=device, dtype=dtype)  # run on selected device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            
            scores = model(x) #computer forward model    
                    
            _, preds = scores.max(1)# max returns both max values and indices, we discard the values
                                    # preds will contain the indices of the classes with highest score (recall the index represents the class)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples * 100
    return acc

In [20]:
class CNN(nn.Module): 
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=n_channels, out_channels=64, kernel_size=3, padding=1, bias=True)
        nn.init.kaiming_normal_(self.conv1.weight)
        
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1, bias=True)
        nn.init.kaiming_normal_(self.conv2.weight)
        
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1, bias=True)
        nn.init.kaiming_normal_(self.conv3.weight)
        
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1, bias=True)
        nn.init.kaiming_normal_(self.conv4.weight)
        
        self.conv5 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1, bias=True)
        nn.init.kaiming_normal_(self.conv5.weight)
        
        self.fc = nn.Linear(in_features=64*93*120, out_features=2, bias=True)
        nn.init.kaiming_normal_(self.fc.weight)
        
        self.m = nn.BatchNorm2d(64, affine=False)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        """x = F.relu(self.m(self.conv4(x)))
        x = F.relu(self.m(self.conv5(x)))"""
        x = x.view(-1, 64*93*120)
        scores = self.fc(x)
        
        return scores

In [21]:
def train(model, optimizer, criterion, num_epochs=5):
    
    model = model.to(device=device)
    for e in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(loader_train, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device=device), labels.to(device=device)
            
            
            pass
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()

            optimizer.step()         
    
        acc = check_accuracy(loader_test, model)
        print(loss.item())
        print('Epoch: %i Accuracy: %f %%' % (e, acc))
                    
learning_rate = 3.5e-7
num_epochs = 5

model = CNN()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [22]:
train(model, optimizer, criterion, num_epochs=num_epochs)

0.6429606676101685
Epoch: 0 Accuracy: 71.929825 %
0.6433526873588562
Epoch: 1 Accuracy: 71.491228 %
0.5661157369613647
Epoch: 2 Accuracy: 72.076023 %
0.7867354154586792
Epoch: 3 Accuracy: 72.953216 %
0.6707269549369812
Epoch: 4 Accuracy: 72.222222 %


In [23]:
acc = check_accuracy(loader_train, model)
print('Accuracy: %f %%' % (acc))

Accuracy: 74.107143 %
